# Initial Setup

- import various helpers, load data, select reviews by status and category

In [1]:
import sys
sys.path.append('../')

from application.name_obj_classes import PubName, PersonName, remove_punct

from application.review_obj_class import ReviewObj

from application.text_preprocessing import preprocess_text

import os
import pandas as pd
import re
from collections import Counter
import numpy as np
import pickle
from nltk.metrics import edit_distance
%pprint

Pretty printing has been turned OFF


In [2]:
from database import *
import database.models as models

# load full text from db
aps_details_single = models.Review().query.filter(models.Review.status.in_(('needs_crosscheck', 'done'))).filter(models.Review.review_type == 'single_focus').all()

len(aps_details_single)

titles = [i.reviewed_book_title for i in aps_details_single if i.reviewed_book_title != '' and i.reviewed_book_title is not None]
len(titles)

561

In [4]:
reviews_parsed = [ReviewObj(i.record_id, i.full_text) for i in aps_details_single if i.reviewed_book_title != '' and i.reviewed_book_title is not None]
#reviews_parsed[0].cleaned_text
#reviews_parsed[0].cleaned_toks[:10]
len(reviews_parsed)

558

In [36]:
from fuzzysearch import find_near_matches
from nltk import word_tokenize

all_matches = {}
befores = []
afters = []
for e,i in enumerate(reviews_parsed):
    t = titles[e]
    #base fuzziness on length of pubname
    if len(t) < 5:
        fuzz=0
    elif len(t) > 5 and len(t) < 10:
        fuzz=1
    elif len(t) > 9 and len(t) < 20:
        fuzz=2
    else:
        fuzz=3
    matches = find_near_matches(t, i.cleaned_text, max_l_dist=fuzz)
    if len(matches) > 0:
        match_strings = [i.cleaned_text[m.start:m.end] for m in matches]
        last = 0
        next_m = 1
        
        for x, m in enumerate(matches):
            before = i.cleaned_text[last:m.start]
            if next_m >= len(matches):
                after = i.cleaned_text[m.end:]
            else:
                after = i.cleaned_text[m.end: matches[next_m].start]
            
            
            last = m.end+1
            next_m = x+2
            
            #lowercase and remove punctuation tokens
            before_tokens = [c.lower() for c in word_tokenize(before) if c.isalpha()]
            after_tokens = [c.lower() for c in word_tokenize(after) if c.isalpha()]
            befores.append(before_tokens)
            afters.append(after_tokens)        
    
        #for i in match_strings:
            #try:
                #all_matches[t][i] +=1
            #except:
                #all_matches[t] = {}
                #all_matches[t][i] = 1
befores[2]

['a', 'bone', 'of', 'contention', 'is', 'a', 'stroke', 'of', 'wit', 'it', 'does', 'however', 'rightly', 'describe', 'the']

In [37]:
afters[2]

['of', 'the', 'past', 'and', 'vividly', 'writes', 'in', 'a', 'phrase', 'her', 'political', 'history', 'until', 'the', 'triumph', 'of', 'the', 'union', 'armies', 'and', 'the', 'close', 'of', 'the', 'civil', 'war']

In [63]:
from collections import Counter 
bigrams = []
for tokens in befores:
    if len(tokens) > 3:
        bigrams.append("_".join(tokens[-2:]))
Counter(bigrams).most_common(30)

[('of_the', 6), ('volume_of', 5), ('edition_of', 3), ('life_of', 3), ('chapter_of', 3), ('law_of', 3), ('egypt_explor', 3), ('the_h', 3), ('story_of', 3), ('the_title', 2), ('volume_called', 2), ('this_of', 2), ('literary_history', 2), ('of_which', 2), ('original_character', 2), ('is_not', 2), ('in_which', 2), ('first_novel', 2), ('latest_novel', 2), ('study_of', 2), ('her_godfa', 2), ('the_m', 2), ('the_i', 2), ('the_egypti', 2), ('of_r', 2), ('at_le', 2), ('the_v', 2), ('seven_li', 2), ('such_d', 2), ('give_us', 2)]

In [53]:
# the first chapter of
# chapter of
# his latest
# of the 
# volume of
# edition of
# volume called 
# first novel 
# study of
# of which 
# latest novel
# the author of 
# the manner in which 
# with the title 
# titled?
# book 
# novel 
# entitled
# is, it is, book is, what is
# life of

In [48]:
bigrams = []
for tokens in afters:
    if len(tokens) > 3:
        bigrams.append("_".join(tokens[0:2]))
Counter(bigrams).most_common(20)

[('is_a', 16), ('of_the', 9), ('edited_by', 8), ('by_the', 8), ('in_the', 7), ('from_the', 5), ('by_george', 5), ('as_a', 4), ('it_is', 4), ('and_the', 4), ('by_john', 4), ('by_charles', 4), ('is_an', 4), ('by_edward', 3), ('a_study', 3), ('at_the', 3), ('is_the', 3), ('by_edgar', 3), ('in_which', 3), ('n_by', 3)]

In [ ]:
#"[title] is a", "[title] is an", "[title] by [author]", 
#"[title] in the", "[title] from the", "[title] is the title"
#[title] is a novel", [title] is a story"